Author: Liu Chang

Description: 

This notebook will demonstrate CNN+RNN hybrid model performance 

Date:
Week 9

In [1]:
import os.path as path
import pickle
import torch
from torch import nn
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

data_path='/raid5/liuchang/quick_draw_output'

with open(path.join(data_path, 'train_X'), 'rb') as f:
    train_X = pickle.load(f)

with open(path.join(data_path, 'train_Y'), 'rb') as f:
    train_Y = pickle.load(f)

with open(path.join(data_path, 'test_X'), 'rb') as f:
    test_X = pickle.load(f)
with open(path.join(data_path, 'test_Y'), 'rb') as f:
    test_Y = pickle.load(f)

In [2]:
labels_count=len(set(test_Y))
print(labels_count)

4


In [3]:
def unpack(x,max_strock,max_len):
    x_new=torch.zeros(torch.Size([len(x),max_strock,3,max_len]))
    for i,item in enumerate(x):
        for j,strock in enumerate(item):
            #print(strock)
            
            #print(len(strock[0]))
            strock=torch.Tensor(strock)
            #print(x_new[i,j,0,:len(strock[0])].shape)
            #print(strock[0].shape)
            x_new[i,j,0,:len(strock[0])]=strock[0]
            x_new[i,j,1,:len(strock[0])]=strock[1]
        x_new[i,0,2,0]=len(item)
    return x_new

def get_max_len(x,xx):
    max_len=0
    max_strock=0
    for i,item in enumerate(x):
        max_strock=max(max_strock,len(item))
        #print(max_strock)
        for j,strock in enumerate(item):
            max_len=max(max_len,len(strock[0])) 
    for i,item in enumerate(xx):
        max_strock=max(max_strock,len(item))
        #print(max_strock)
        for j,strock in enumerate(item):
            max_len=max(max_len,len(strock[0])) 
    return max_strock,max_len

max_strock,max_len=get_max_len(train_X,test_X)
print(max_strock,max_len)
train_X=unpack(train_X,max_strock,max_len)
test_X=unpack(test_X,max_strock,max_len)

59 542


In [4]:
def get_acc(test_X,test_Y):
    cur_len=0
    acc=0
    while cur_len<test_X.shape[0]:
        model.zero_grad()
        minibatch_X=test_X[cur_len:cur_len+batch_size].cuda()
        #print(minibatch_X.shape)
        minibatch_Y=test_Y[cur_len:cur_len+batch_size].cuda()
        y_predict=model(minibatch_X)
        #print(y_predict)
        y_predict=torch.argmax(y_predict,dim=1)
        #print(y_predict.shape)
        cur_len+=batch_size
        acc+=(y_predict==minibatch_Y).sum().item()
    return acc/cur_len

In [5]:
class CNN(nn.Module):
    def __init__(self,output_size):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(
                in_channels=3,      # input height
                out_channels=6,    # n_filters
                kernel_size=3      # filter size
            ),
            nn.ReLU(),    # activation
            nn.MaxPool1d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(
             nn.Conv1d(6,32, 20),
             nn.ReLU(),  # activation
             nn.MaxPool1d(kernel_size=10)
         )
        self.out = nn.Linear(800, output_size)
        #self.out2 = nn.Linear(3*(max_len), output_size)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        #x=self.conv3(x)
        x = x.view(x.size(0), -1)
        #print(x.shape)
        output = self.out(x)
        #output=self.out2(x)
        return output
    
class CRNN(nn.Module):
    def __init__(self):
        super(CRNN, self).__init__()
        self.middle_size=middle_size=40
        self.gru=nn.GRU(input_size=middle_size,hidden_size=5,num_layers=2,bias=False)
        # self.gru=nn.GRU(input_size=3,hidden_size=5,num_layers=1,bias=True)
        self.cnn=CNN(output_size=middle_size)
        self.out=nn.Softmax(dim=-1)
    def forward(self, x):
        x=x.view(-1,3,max_len)
        
        #print(x.shape)
        xx = self.cnn(x)
        xx=xx.view(-1,max_strock,self.middle_size).transpose(0,1)
        output,_ = self.gru(xx)
        #print(output.shape)
        to_send=torch.Tensor(torch.Size([xx.shape[1],5])).cuda()
        for i in range(xx.shape[1]):
            to_send[i]=output[int(x[i*max_strock,2,0])-1,i]
        output = self.out(to_send)
        return output

In [ ]:
import torch.optim as optim

model=CRNN().cuda()
batch_size=256
loss=torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
EPOCH=256
train_Y=torch.LongTensor(train_Y)
test_Y=torch.LongTensor(test_Y)
for i in range(EPOCH):
    total_loss=0
    cur_len=0
    while cur_len<train_X.shape[0]:
        model.zero_grad()
        minibatch_X=train_X[cur_len:cur_len+batch_size].cuda()
        #print(minibatch_X.shape)
        minibatch_Y=train_Y[cur_len:cur_len+batch_size].cuda()
        y_predict=model(minibatch_X)
        
        #print(y_predict.shape,minibatch_Y.shape)
        #print(loss,y_predict,minibatch_Y)
        output_loss=loss(y_predict,minibatch_Y)
        output_loss.backward()
        optimizer.step()
        cur_len+=batch_size
        total_loss+=output_loss.item()
        #print("Processing: {}/{}, loss={}".format(cur_len,train_X.shape[0],output_loss.item()))
    if i%5==0:
        print("EPOCH {}/{}, loss={},acc={}".format(i,EPOCH,total_loss,get_acc(test_X,test_Y)))

EPOCH 0/256, loss=162.04698705673218,acc=0.34736689814814814
EPOCH 5/256, loss=145.22923457622528,acc=0.7037037037037037
EPOCH 10/256, loss=139.502117395401,acc=0.7259837962962963
EPOCH 15/256, loss=138.01459872722626,acc=0.7563657407407407
EPOCH 20/256, loss=138.31257164478302,acc=0.7160011574074074
EPOCH 25/256, loss=136.60594165325165,acc=0.7877604166666666
EPOCH 30/256, loss=137.85991168022156,acc=0.7896412037037037
EPOCH 35/256, loss=135.61309587955475,acc=0.7954282407407407
EPOCH 40/256, loss=134.78832042217255,acc=0.8171296296296297
EPOCH 45/256, loss=134.12455427646637,acc=0.8198784722222222


In [ ]:
get_acc(test_X,test_Y)